In [2]:
from clamv import TestDataGen, Resizer, FormatConverter, MultiDimensionalModel
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import numpy as np

# Data

In [3]:
resize = Resizer(50, 60)
rgb2yuv = FormatConverter(source='RGB', dest='YUV', flip_axis=True)

In [3]:
data = [(rgb2yuv(resize(x)), y) for x, y in TestDataGen(path='../recognition-pipeline/data', numeric_y=True)]
X, y = zip(*data)
X = np.array(X).reshape(len(X), 3, -1)
y = np.array(y)

# Test

In [9]:
from sklearn.model_selection import cross_val_score

# EigenFaces

In [6]:
EigenFaces = Pipeline([
    ('eigenfaces', MultiDimensionalModel(PCA(n_components=150), dimensions=3)),
    ('classify', SVC(kernel='linear', class_weight='balanced'))
])

In [7]:
%time cross_val_score(EigenFaces, X, y, cv=3)

Wall time: 31.3 s


array([ 0.76093943,  0.85481078,  0.595692  ])

# FisherFaces

In [10]:
FisherFaces = Pipeline([
    ('fisherfaces', MultiDimensionalModel(LDA(n_components=150), dimensions=3)),
    ('classify', SVC(kernel='linear', class_weight='balanced'))
])

In [11]:
%time cross_val_score(FisherFaces, X, y, cv=3)

Wall time: 2min 28s


array([ 0.62941904,  0.76255256,  0.53107205])

# Using New Data
# FisherFaces

In [95]:
X = np.load('clmv_X.npy')
y = np.load('clmv_y.npy')

names = np.load('clmv_names.npy')

In [96]:
X_yuv = []

rgb2yuv = FormatConverter(source='RGB', dest='YUV', flip_axis=True)
for x in X:
    X_yuv.append(rgb2yuv(x))
    
X_yuv = np.array(X_yuv)

In [97]:
X_yuv = X_yuv.reshape(len(X_yuv), 3, -1)

In [147]:
%time cross_val_score(EigenFaces, X_yuv, y, cv=3)

/Users/tom/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:455: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
/Users/tom/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:455: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


CPU times: user 1min 37s, sys: 8.9 s, total: 1min 46s
Wall time: 51.4 s


array([ 0.76221498,  0.76651982,  0.78764045])

## Doing a train test split

In [162]:
from sklearn.model_selection import train_test_split

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X_yuv, y, random_state=42)

In [164]:
FisherFaces.fit(X_train, y_train)

/Users/tom/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:455: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Pipeline(steps=[('eigenfaces', MultiDimensionalModel(dimensions=None,
           models=[PCA(copy=True, iterated_power='auto', n_components=150, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False), PCA(copy=True, iterated_power='auto', n_components=150, random_state=None,
  svd_solver='auto', tol=...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [165]:
from sklearn.metrics import accuracy_score
accuracy_score(FisherFaces.predict(X_test), y_test)

0.78529411764705881